<a href="https://colab.research.google.com/github/georgesteve/Research-Assitant-ML-ROJ/blob/main/Semana_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#[Forecasting Global Ionospheric TEC Using Deep Learning Approach](https://doi.org/10.1029/2020SW002501)

In [8]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [9]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [15]:
ls

 0001.jpg
 85192060-Los-cincuenta-anos-de-Jicamarca.pdf
'98 FORMATO  DEDICATORIA APG.doc (1).docx'
'ATRAD presentación 1 ROJ.gslides'
 Classroom/
'Colab Notebooks'/
'COMPENDIO FAJARDO SORIA'/
'Copia de Checklist para discusiones públicas.gsheet'
'Copia de CitiesExt.csv.gsheet'
'Copia de Copia de Copia de TESIS FAJARDO SORIA GEORGE Presentación Final.gslides'
'Copia de deltaV station keeping .gdoc'
'Copia de Manual Sesiones de Prueba de Amanecidas Hackspace.gdoc'
'Copia de PlayersExt.csv.gsheet'
'Copia de Process Flow Diagram Template.gdraw'
'Copia de TESIS FAJARDO SORIA GEORGE Presentación Final (1).gslides'
'Copia de TESIS FAJARDO SORIA GEORGE Presentación Final.gslides'
'Copia de yqw-xmbf-pkw - 3 de dic. de 2020.gjam'
'coreupgrade retos'/
 Cuentas.gsheet
 curso-frances.pdf
'Documento sin título (1).gdoc'
'Documento sin título (2).gdoc'
'Documento sin título.gdoc'
'ECI TESIS FAJARDO SORIA GEORGE Presentación Final (1).gslides'
'ECI TESIS FAJARDO SORIA GEORGE Presentación Fin

Este paper utiliza RNN con arquitecturas LSTM para predecir condiciones de TEC.


In [ ]:
https://github.com/georgesteve/Research-Assitant-ML-ROJ.git

In [17]:
import pandas
URL = "https://raw.githubusercontent.com/georgesteve/Research-Assitant-ML-ROJ/main/gfajardo_uNz2xU-2020-2.csv"
Data = pandas.read_csv(URL) 
Data.head() 

,Unnamed: 0,Año,Day,Tiempo,PRN,S4,Az,Elv
0,0,20,306,22,3,0.10,300.9,24.6
1,1,20,306,22,8,0.05,199.5,47.2
2,2,20,306,22,10,0.40,147.1,14.3
3,3,20,306,22,11,0.08,210.6,26.2
4,4,20,306,22,16,0.04,342.4,42.3


In [19]:
Data['Año']

0         20
1         20
2         20
3         20
4         20
          ..
452982    20
452983    20
452984    20
452985    20
452986    20
Name: Año, Length: 452987, dtype: int64